In [1]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./pdf/IIA Implementation-Guides.pdf")
# loader = PyPDFLoader("./pdf/IIA Standards - Audit Manual.pdf")
documents = loader.load()

In [2]:
# documents

In [3]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [4]:
len(docs)

203

In [5]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\yiclive\Anaconda3\envs\py311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain_community.vectorstores import Chroma

db = Chroma.from_documents(docs, embedding_function, persist_directory="chroma_db")

In [8]:
query = "What are the steps involved in audit engagement planning?"
docs = db.similarity_search(query)

In [9]:
docs[0]

Document(page_content='International Standards for the Professional Practice of Internal Auditing  (Standards)  \nRevised:  October 2016  Page 16 of 25 \nEffective: January 2017   \n© 2016  The Institute of Internal Auditors  2220.A2 – If significant consulting opportunities arise during an assurance engage ment, a \nspecific written understanding as to the objectives, scope, respective responsibilities, and \nother expectations should be reached and the results of the consulting engagement \ncommunicated in accordance with consulting standards.  \n2220.C1  – In perfor ming consulting engagements, internal auditors must ensure that the \nscope of the engagement is sufficient to address the agreed -upon objectives. If internal \nauditors develop reservations about the scope during the engagement, these reservations \nmust be discussed with the client to determine whether to continue with the engagement.  \n2220.C2  – During consulting engagements, internal auditors must address controls

In [10]:
docs[1]

Document(page_content="International Standards for the Professional Practice of Internal Auditing  (Standards)  \nRevised:  October 2016  Page 15 of 25 \nEffective: January 2017   \n© 2016  The Institute of Internal Auditors  2201.A1  – When planning an engagement for parties outside the organization, internal \nauditors must establish a written understanding with them about objectives, scope,  \nrespective responsibilities, and other expectations, including restrictions on distribution of \nthe results of the engagement and access to engagement records.  \n2201.C1  – Internal auditors must establish an understanding with consulting engagement \nclients about objectives, scope, respective responsibilities, and other client expectations. \nFor significant engagements, this understanding must be documented.  \n2210 – Engagement Objectives  \nObjectives must be established for each engagement.  \n2210.A1  – Internal auditors must conduct a preliminary assessment of the risks relevant to \

In [11]:
docs[2]

Document(page_content='International Standards for the Professional Practice of Internal Auditing  (Standards)  \nRevised:  October 2016  Page 17 of 25 \nEffective: January 2017   \n© 2016  The Institute of Internal Auditors  Sufficient information is factual, adequate, and convincing so that a prudent, informed person \nwould reach the same conclusions as the auditor. Reliable information is the best attainable \ninformation through the use of appropriate engagement techniques. Relevant information \nsupports engagement observations and recommendations and is consistent with the objectives \nfor the engagement. Useful information helps the organization meet its goals.  \n2320 – Analysis and Evaluation  \nInternal auditors must base conclusions and engagement results on appropriate analyses and \nevaluations.  \n2330 – Documenting  Information  \nInternal auditors must document sufficient, reliable, relevant, and useful information to support \nthe engageme nt results and conclusions. 

In [19]:
db.as_retriever(search_kwargs={"filter":{"source":"./pdf/IIA Implementation-Guides.pdf"}})

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000002B3CE042010>, search_kwargs={'filter': {'source': './pdf/IIA Implementation-Guides.pdf'}})

In [16]:
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"filter":{"source":"./pdf/IIA Implementation-Guides.pdf"}})
)


In [15]:
# retriever=db.as_retriever(search_kwargs={"filter":{"source":"./pdf/IIA Standards - Audit Manual.pdf"}})
qa.invoke("What is the chief auditor's role?")

{'query': "What is the chief auditor's role?",
 'result': "The chief auditor's role is to effectively manage the internal audit activity to ensure it adds value to the organization. This includes achieving the purpose and responsibility outlined in the internal audit charter, conforming with the Standards and Code of Ethics, considering trends and emerging issues that could impact the organization, and providing relevant assurance. The chief auditor is also responsible for establishing a risk-based plan for the internal audit activity, consulting with senior management and the board to understand the organization's strategies, objectives, and risks, and adjusting the plan as necessary. Additionally, the chief auditor communicates and interacts directly with the board, monitors the disposition of results communicated to management, and discusses any unacceptable levels of risk accepted by management with senior management and the board."}

In [18]:
# retriever=db.as_retriever(search_kwargs={"filter":{"source":"./pdf/IIA Implementation-Guides.pdf"}})
qa.invoke("What is the chief auditor's role?")

{'query': "What is the chief auditor's role?",
 'result': 'The chief audit executive (CAE) is responsible for overseeing the internal audit activity within an organization. Their role includes ensuring the independence and objectivity of the internal audit function, managing the internal audit staff, developing and implementing audit plans, and providing assurance and consulting services to the organization. The CAE is also responsible for assessing the competency of internal auditors, developing policies and procedures for the internal audit activity, and promoting a culture of integrity and ethical behavior within the organization.'}

In [20]:
qa.invoke("What is the chief auditor's role?")['result']

'The chief audit executive (CAE) is responsible for overseeing the internal audit activity within an organization. Their role includes ensuring the independence and objectivity of the internal audit function, managing the internal audit staff, and providing assurance and consulting services to the organization. The CAE is also responsible for assessing the competency of individual internal auditors, developing policies and procedures for the internal audit activity, and promoting a culture of integrity within the organization. Additionally, the CAE may have roles and responsibilities outside of internal auditing, such as compliance or risk management activities, which need to be managed to avoid impairments to independence or objectivity.'